<a href="https://colab.research.google.com/github/vyassuyog1/Suyog/blob/master/SentimentClassificationBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import torch
from sklearn.model_selection import train_test_split


#Loading Data

In [2]:
df=pd.read_csv("/content/smile-annotations-final.csv",names=['id','text','category'])
df.set_index('id',inplace=True)
df

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy
...,...,...
613678555935973376,MT @AliHaggett: Looking forward to our public ...,happy
613294681225621504,@britishmuseum Upper arm guard?,nocode
615246897670922240,@MrStuchbery @britishmuseum Mesmerising.,happy


In [3]:
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|angry               2
sad|disgust             2
sad|disgust|angry       1
Name: category, dtype: int64

In [4]:
'''df.drop(labels=[i for i in df.index if df.loc[i]['category']=='happy|surprise' or df.loc[i]['category']=='disgust|angry'
                                                            or df.loc[i]['category']=='happy|sad' or df.loc[i]['category']=='sad|angry' 
                                                            or df.loc[i]['category']=='sad|disgust'
                                                            or df.loc[i]['category']=='sad|disgust|angry'],inplace=True)'''

"df.drop(labels=[i for i in df.index if df.loc[i]['category']=='happy|surprise' or df.loc[i]['category']=='disgust|angry'\n                                                            or df.loc[i]['category']=='happy|sad' or df.loc[i]['category']=='sad|angry' \n                                                            or df.loc[i]['category']=='sad|disgust'\n                                                            or df.loc[i]['category']=='sad|disgust|angry'],inplace=True)"

In [5]:
df=df[~df.category.str.contains('\|')]                               #deleting categoreis having 1+ emotion i.e have "|" in it
df=df[df.category!='nocode']

In [6]:
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [7]:
label_dict={k:v for v,k in enumerate(df.category.unique())}
label_dict                                                               #dict for emotions and numeric labels

{'angry': 2,
 'disgust': 3,
 'happy': 0,
 'not-relevant': 1,
 'sad': 4,
 'surprise': 5}

In [8]:
#df['label']=[label_dict[i] for i in df.category]
df['label']=df.category.replace(label_dict)                                    #make new label column using replace
df

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0
...,...,...,...
611258135270060033,@_TheWhitechapel @Campaignforwool @SlowTextile...,not-relevant,1
612214539468279808,“@britishmuseum: Thanks for ranking us #1 in @...,happy,0
613678555935973376,MT @AliHaggett: Looking forward to our public ...,happy,0


In [9]:
X_train,X_val,y_train,y_val=train_test_split(df.index.values,                              #train_test_split using stratify with labels
                                             df.label.values,                              #stratify divides every label in same proportion
                                             test_size=0.15,                                                  #here 85,15 %
                                             random_state=17,
                                             stratify=df.label.values)

In [10]:
df['data_type']=['not_set'] * df.shape[0]                                           #column indication row from train or val set 
df

,text,category,label,data_type
id,,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,not_set
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,not_set
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,not_set
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,not_set
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,not_set
...,...,...,...,...
611258135270060033,@_TheWhitechapel @Campaignforwool @SlowTextile...,not-relevant,1,not_set
612214539468279808,“@britishmuseum: Thanks for ranking us #1 in @...,happy,0,not_set
613678555935973376,MT @AliHaggett: Looking forward to our public ...,happy,0,not_set


In [11]:
df.loc[X_train,"data_type"]='train'
df.loc[X_val,"data_type"]='val'
df


,text,category,label,data_type
id,,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,train
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,train
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,train
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,train
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,train
...,...,...,...,...
611258135270060033,@_TheWhitechapel @Campaignforwool @SlowTextile...,not-relevant,1,train
612214539468279808,“@britishmuseum: Thanks for ranking us #1 in @...,happy,0,train
613678555935973376,MT @AliHaggett: Looking forward to our public ...,happy,0,train


In [12]:
df.groupby(['category','label','data_type']).count()

text
category     label data_type      
angry        2     train        48
                   val           9
disgust      3     train         5
                   val           1
happy        0     train       966
                   val         171
not-relevant 1     train       182
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

#Using Bert Tokenizer

In [13]:
!pip install transformers

In [14]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [15]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased',
                                        do_lower_case=True)                        #uncased- as it is not lowered and 
                                                                                   #it uses pretrained tokens(i.e from_pretrained)

In [16]:
encoded_data_train=tokenizer.batch_encode_plus(df[df.data_type=='train'].text.values,                        #for train data
                                               add_special_tokens=True,                          # add spl tokens (start , end)
                                               return_attention_mask=True,                        # the actual length of sentence
                                               pad_to_max_length=True,                           #pad till max len(to get same i/p size)
                                               max_length=256,
                                               return_tensors='pt')                             #pt=pyTorch

encoded_data_val=tokenizer.batch_encode_plus(df[df.data_type=='val'].text.values,
                                               add_special_tokens=True,
                                               return_attention_mask=True,
                                               pad_to_max_length=True,
                                               max_length=256,
                                               return_tensors='pt')

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [17]:
input_ids_train=encoded_data_train['input_ids']                                  #above task returns dictionary having these keys
attention_masks_train=encoded_data_train['attention_mask']                       #input ids have the tensors of tokens of every word
labels_train=torch.tensor(df[df.data_type=='train'].label.values)                #making tensor of labels

input_ids_val=encoded_data_val['input_ids']
attention_masks_val=encoded_data_val['attention_mask']
labels_val=torch.tensor(df[df.data_type=='val'].label.values)

In [18]:
dataset_train=TensorDataset(input_ids_train,attention_masks_train,labels_train)   #using tensor dataset

dataset_val=TensorDataset(input_ids_val,attention_masks_val,labels_val)

In [19]:
from transformers import BertForSequenceClassification

In [20]:
model= BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',                                              #basic form of BERT easy to handle
    num_labels=len(label_dict),                                       #number of target classes i.e emotions
    output_attentions=False,                                          #output ate each steps-False
    output_hidden_states=False                                        #output of hidden states
                                                                      # the fine tuning step(i.e (i think)  tuning the model 
                                                                      #as per our requirements e.g num labels etc)
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

#Creating Data Loaders

In [21]:
from torch.utils.data import DataLoader, RandomSampler,SequentialSampler

In [35]:
batch_size=4

dataloader_train=DataLoader(dataset_train,sampler=RandomSampler(dataset_train),             #RandomSampler so that model doesn't learn anything based on sequence of inputs
                            batch_size=batch_size)

dataloader_val=DataLoader(dataset_val,sampler=RandomSampler(dataset_val),             #RandomSampler so that model doesn't learn anything based on sequence of inputs
                            batch_size=32)


#Setting up Optimizer and Scheduler

In [23]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [24]:
optimizer=AdamW(model.parameters(),
                lr=2e-5,  #to 5e-5   ,1e-5
                eps=1e-8
)

In [25]:
epochs=10
scheduler=get_linear_schedule_with_warmup(optimizer,
                                          num_warmup_steps=0,
                                          num_training_steps=len(dataloader_train) * epochs
                                          )

#Stating the Performance matrics

In [26]:
import numpy as np
from sklearn.metrics import f1_score
import torch.nn

In [27]:
def f1_score_func(preds,labels):
  pred_flat=np.argmax(preds,axis=1).flatten()                   #flatten-collapses any dimention array into 1-D (though originally more or less)
  labels_flat=labels.flatten()

  return f1_score(labels_flat,pred_flat,average="weighted")


In [28]:
def accuracy_per_class(preds,labels):
  label_dict_inverse={v:k for k,v in label_dict.items()}

  pred_flat=np.argmax(preds,axis=1).flatten()                   #flatten-collapses any dimention array into 1-D (though originally more or less)
  labels_flat=labels.flatten()

  for label in np.unique(labels):
    y_pred=pred_flat[labels_flat==label]
    y_true=labels_flat[labels_flat==label]
    print(f"Class: {label_dict_inverse[label]}")
    print(f"Accuracy: {str(len(y_pred[y_pred==label])) +'/' +str(len(y_true))}\n")

#Training the model

In [29]:
import random

seed_val=17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [30]:
device=torch.device('cuda' if torch.cuda.is_available() else "cpu")
model.to(device)

print(device)

cuda


In [31]:
def evaluate(dataloader):
  model.eval()
  loss_val_total=0
  predictions,true_vals=[],[]

  for batch in dataloader:
    batch=tuple(b.to(device) for b in batch)

    inputs= {                                                                    #batch is in the form of dict having keys input ids,att. mask,labels
        'input_ids':     batch[0],
        'attention_mask':batch[1],
        'labels':        batch[2]
    }
    with torch.no_grad():
      output=model(**inputs)

    loss=output[0]
    logits=output[1]
    loss_val_total+=loss.item()

    logits=logits.detach().cpu().numpy()
    label_ids=inputs['labels'].cpu().numpy()

    predictions.append(logits)
    true_vals.append(label_ids)

  loss_val_avg=loss_val_total/len(dataloader)

  predictions=np.concatenate(predictions,axis=0)
  true_vals=np.concatenate(true_vals,axis=0) 

  return loss_val_avg,predictions,true_vals



In [36]:
for epoch in tqdm(range(1,epochs+1)):
  model.train()

  loss_train_total=0

  progress_bar=tqdm(dataloader_train,desc='Epoch {:1d}'.format(epoch),
                    leave=False,disable=False)
  
  for batch in progress_bar:

    model.zero_grad()

    batch=tuple(b.to(device) for b in batch)

    inputs= {
        'input_ids':     batch[0],
        'attention_mask':batch[1],
        'labels':        batch[2]
    }

    output=model(**inputs)

    loss=output[0]
    loss_train_total+=loss.item()
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)               #clips the gradients to (1.0)  avoid very  small and very large gradients

    optimizer.step()
    scheduler.step()

    progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})                 #postfix to display it after epoch progressbar

  torch.save(model.state_dict(),f'/content/BERT_ft_epoch{epoch}.model')

  tqdm.write('\n Epoch {epoch}')
  loss_train_avg=loss_train_total/len(dataloader_train)
  tqdm.write(f'Training Avg Loss: {loss-loss_train_avg}')

  val_loss,predictions,true_vals=evaluate(dataloader_val)
  val_f1=f1_score_func(predictions,true_vals)
  tqdm.write(f'Validation Avg Loss :{val_loss}')
  tqdm.write(f'F1 Score (weighted): {val_f1}')


 Epoch {epoch}
Training Avg Loss: -0.060414597392082214
Validation Avg Loss :0.5513768174818584
F1 Score (weighted): 0.8605155066510909



 Epoch {epoch}
Training Avg Loss: -0.05788002535700798
Validation Avg Loss :0.5524002271039146
F1 Score (weighted): 0.8605155066510909



 Epoch {epoch}
Training Avg Loss: -0.06387128680944443
Validation Avg Loss :0.5525680737836021
F1 Score (weighted): 0.8605155066510909



 Epoch {epoch}
Training Avg Loss: -0.063997782766819
Validation Avg Loss :0.5515446322304862
F1 Score (weighted): 0.8605155066510909



 Epoch {epoch}
Training Avg Loss: -0.060684796422719955
Validation Avg Loss :0.5504253485373088
F1 Score (weighted): 0.8605155066510909



 Epoch {epoch}
Training Avg Loss: -0.02735835686326027
Validation Avg Loss :0.5534381419420242
F1 Score (weighted): 0.8605155066510909



 Epoch {epoch}
Training Avg Loss: 0.005388602614402771
Validation Avg Loss :0.552752312006695
F1 Score (weighted): 0.8605155066510909



 Epoch {epoch}
Training Avg Loss: -0.061075445264577866
Validation Avg Loss :0.549749527525689
F1 Score (weighted): 0.8605155066510909



 Epoch {epoch}
Training Avg Loss: -0.06254036724567413
Validation Avg Loss :0.5507485036339078
F1 Score (weighted): 0.8605155066510909



 Epoch {epoch}
Training Avg Loss: -0.05493408441543579
Validation Avg Loss :0.550833421094077
F1 Score (weighted): 0.8605155066510909



In [37]:
accuracy_per_class(predictions,true_vals)

Class: happy
Accuracy: 167/171

Class: not-relevant
Accuracy: 18/32

Class: angry
Accuracy: 7/9

Class: disgust
Accuracy: 0/1

Class: sad
Accuracy: 1/5

Class: surprise
Accuracy: 2/5



In [38]:
torch.cuda.get_device_name(0)

'Tesla T4'